link:

http://insider.espn.com/nba/hollinger/statistics

In [1]:
import pandas as pd
import csv

In [19]:
from selenium.webdriver import Firefox
from bs4 import BeautifulSoup

url = 'https://www.teamrankings.com/nba/player-stat/nba-efficiency?season_id=216'
driver = Firefox()
driver.get(url)

In [20]:
soup = BeautifulSoup(driver.page_source)

In [6]:
headers = soup.find(_class='colhead')

In [24]:
headers = soup.find('thead').find('tr').find_all('th')
headers = [i.text for i in headers]

In [25]:
headers

['Rank', 'Player', 'Team', 'Pos', 'Value']

In [71]:
with open('top_per_0607.csv', 'w') as file:
    csv_writer = csv.writer(file, delimiter=',')
    csv_writer.writerow([header for header in headers])
#     for i in range(9):
        
    soup = BeautifulSoup(driver.page_source) 
    players = soup.find('table').find_all('tr')[1:]
    for player in players:
        csv_writer.writerow( 
            [item.text for item in player.find_all('td')]
        )
    
# driver.find_element_by_xpah(r'/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[3]/div/div/a[2]').click()

In [72]:
data = pd.read_csv('top_per_1617.csv')
data.tail()

,Rank,Player,Team,Pos,Value
95,96,Kentavious Caldwell-Pope,Detroit Pistons,SG,12.03
96,97,Courtney Lee,New York Knicks,SG,11.86
97,98,Rondae Hollis-Jefferson,Brooklyn Nets,SF,11.81
98,99,Alex Len,Phoenix Suns,C,11.81
99,100,Jordan Clarkson,Los Angeles Lakers,PG,11.80


Now, then merge the results (left joint) with nba phytho

In [88]:
import glob
files = glob.glob("top_per*csv")

drop_cols = ['Team', 'Pos']
phythro = pd.read_csv('phythro_nba.csv')

In [91]:
for file in files:
    data = pd.read_csv(file)
    data = data.drop(drop_cols, axis=1)
    data.rename(columns={'Value':'Efficiency'}, inplace=True)
    data = data.merge(phythro, how='left', on='Player')
    data.to_csv(file)

Done. Now we have enought data to do analysis.

In [1]:
import pandas as pd
import csv

In [73]:
import re

r = re.compile(r"([\w\s'-.]+), (\w+)")

def get_team(el):
    m = r.match(el)
    if m == None:
        return float('NaN')
    else:
        return(m.group(2))
def get_player(el):
    m = r.match(el)
    if m == None:
        return float('NaN')
    else:
        return(m.group(1))
    

In [148]:
page_num = re.compile(r'(\d+) (\w+) (\d+)')

In [170]:
from selenium.webdriver import Firefox
from bs4 import BeautifulSoup

url = 'http://insider.espn.com/nba/hollinger/statistics'

driver = Firefox()
driver.get(url)

In [151]:
soup = BeautifulSoup(driver.page_source)

In [152]:
headers = soup.find('table').find(class_='colhead').find_all('td')
headers = [item.text for item in headers]
headers

['RK',
 'PLAYER',
 'GP',
 'MPG',
 'TS%',
 'AST',
 'TO',
 'USG',
 'ORR',
 'DRR',
 'REBR',
 'PER',
 'VA',
 'EWA']

In [189]:

page = 1


for year in range(2000, 2021):
    url = 'http://insider.espn.com/nba/hollinger/statistics/_/page/1/year/' + str(year)
    page_state = soup.find(class_='page-numbers').text
    max_page = int(page_num.match(page_state).group(3))

    with open('full_per_' + str(year) + '.csv', 'w') as file:

        csv_writer = csv.writer(file, delimiter=',')
        csv_writer.writerow([header for header in headers])


        for page in range(1, max_page+1):
            url = 'http://insider.espn.com/nba/hollinger/statistics/_/page/' + str(page) + '/year/' + str(year)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source) 
            players = soup.find('table').find_all('tr')[1:]
            players = soup.find('table').find_all('tr')

            for player in players[2:]:
                cells = player.find_all('td')
                cells = [item.text for item in cells]
                if cells[0] == 'RK':
                    continue
                else:
                    csv_writer.writerow(cells)


In [149]:
import pandas as pd
d = pd.read_csv('full_per_2019.csv')
d.head(5) # Jokic?

,RK,PLAYER,GP,MPG,TS%,AST,TO,USG,ORR,DRR,REBR,PER,VA,EWA
0,1.0,"Giannis Antetokounmpo, MIL",72,32.8,0.644,18.9,12.0,32.3,7.3,30.0,19.3,30.95,684.4,22.8
1,2.0,"James Harden, HOU",78,36.8,0.616,18.0,11.9,40.8,2.5,17.8,10.0,30.62,839.5,28.0
2,3.0,"Anthony Davis, NO",56,33.0,0.597,14.1,7.2,29.4,9.9,27.5,18.8,30.32,519.7,17.3
3,4.0,"Karl-Anthony Towns, MIN",77,33.1,0.622,12.9,11.9,28.8,10.9,29.3,20.0,26.38,599.6,20.0
4,NaN,"Nikola Jokic, DEN",80,31.3,0.589,26.5,11.3,29.4,9.8,27.6,18.7,26.38,589.7,19.7


In [150]:
d.at[4, 'RK'] = 5
d.head(5)

,RK,PLAYER,GP,MPG,TS%,AST,TO,USG,ORR,DRR,REBR,PER,VA,EWA
0,1.0,"Giannis Antetokounmpo, MIL",72,32.8,0.644,18.9,12.0,32.3,7.3,30.0,19.3,30.95,684.4,22.8
1,2.0,"James Harden, HOU",78,36.8,0.616,18.0,11.9,40.8,2.5,17.8,10.0,30.62,839.5,28.0
2,3.0,"Anthony Davis, NO",56,33.0,0.597,14.1,7.2,29.4,9.9,27.5,18.8,30.32,519.7,17.3
3,4.0,"Karl-Anthony Towns, MIN",77,33.1,0.622,12.9,11.9,28.8,10.9,29.3,20.0,26.38,599.6,20.0
4,5.0,"Nikola Jokic, DEN",80,31.3,0.589,26.5,11.3,29.4,9.8,27.6,18.7,26.38,589.7,19.7


In [151]:
phys = pd.read_csv('nba1819_phys_stats.csv')

In [152]:
d['TEAM'] = d['PLAYER'].apply(get_team)
d['PLAYER'] = d['PLAYER'].apply(get_player)

In [153]:
data = d.merge(phys, how='left', left_on=['PLAYER'], right_on=['Player'])

In [154]:
data[['PLAYER', 'PER', 'RK', 'Height', 'Weight']].to_csv('short_PER_1819.csv')

In [155]:
data[data.Height.isna()]

,RK,PLAYER,GP_x,MPG,TS%_x,AST_x,TO,USG,ORR,DRR,...,GP_y,PTS,REB,AST_y,NetRtg,OREB%,DREB%,USG%,TS%_y,AST%
97,98.0,T.J. Leaf,58,9.0,0.572,10.3,5.2,16.9,9.3,17.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,242.0,Nene,42,13.0,0.552,15.0,6.3,11.7,6.7,18.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,278.0,Juan Hernangomez,70,19.4,0.580,12.4,8.3,12.3,5.1,16.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,324.0,C.J. Miles,53,16.2,0.504,8.9,7.5,18.0,1.7,10.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
data.at[97, 'Height'] = '6-10'
data.at[241, 'Height'] = '6-11'
data.at[277, 'Height'] = '6-9'
data.at[323, 'Height'] = '6-6'

In [157]:
foot = data['Height'].str.extract(r'(\d+)-', expand=False).apply(int)
inches = data['Height'].str.extract(r'-(\d+)', expand=False).apply(int)
data['Height_cm'] = 2.54*(foot * 12 + inches)

In [158]:
data[['PLAYER', 'PER', 'RK', 'Height_cm', 'Weight']].to_csv('short_PER_1819.csv')

In [159]:
data

,RK,PLAYER,GP_x,MPG,TS%_x,AST_x,TO,USG,ORR,DRR,...,PTS,REB,AST_y,NetRtg,OREB%,DREB%,USG%,TS%_y,AST%,Height_cm
0,1.0,Giannis Antetokounmpo,72,32.8,0.644,18.9,12.0,32.3,7.3,30.0,...,27.7,12.5,5.9,12.5,6.6%,27.3%,31.4%,64.4%,29.4%,210.82
1,2.0,James Harden,78,36.8,0.616,18.0,11.9,40.8,2.5,17.8,...,36.1,6.6,7.5,6.3,2.3%,15.7%,39.6%,61.6%,39.4%,195.58
2,3.0,Anthony Davis,56,33.0,0.597,14.1,7.2,29.4,9.9,27.5,...,25.9,12.0,3.9,3.4,8.9%,24.9%,28.8%,59.7%,18.6%,208.28
3,4.0,Karl-Anthony Towns,77,33.1,0.622,12.9,11.9,28.8,10.9,29.3,...,24.4,12.4,3.4,0.9,9.7%,25.7%,28.0%,62.2%,16.9%,213.36
4,5.0,Nikola Jokic,80,31.3,0.589,26.5,11.3,29.4,9.8,27.6,...,20.1,10.8,7.3,5.8,8.9%,24.9%,27.1%,58.9%,36.1%,213.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,357.0,Lance Thomas,46,17.0,0.482,10.2,9.1,12.6,2.7,13.1,...,4.5,2.5,0.6,-10.9,2.3%,11.5%,12.7%,48.2%,5.1%,203.20
357,358.0,Alex Abrines,31,19.0,0.507,10.2,7.1,12.1,0.9,7.8,...,5.3,1.5,0.6,1.6,0.7%,6.7%,12.0%,50.7%,4.7%,198.12
358,359.0,Treveon Graham,35,20.4,0.458,13.5,6.8,12.7,3.4,12.3,...,5.3,3.1,1.0,2.3,2.8%,10.7%,12.3%,45.8%,6.2%,195.58
359,360.0,Frank Ntilikina,43,21.0,0.417,25.6,11.9,17.4,1.4,8.9,...,5.7,2.0,2.8,-8.5,1.2%,8.0%,16.3%,41.7%,19.5%,198.12


In [160]:
d = pd.read_csv('short_PER_1819.csv')
d

,Unnamed: 0,PLAYER,PER,RK,Height_cm,Weight
0,0,Giannis Antetokounmpo,30.95,1.0,210.82,242.0
1,1,James Harden,30.62,2.0,195.58,220.0
2,2,Anthony Davis,30.32,3.0,208.28,253.0
3,3,Karl-Anthony Towns,26.38,4.0,213.36,248.0
4,4,Nikola Jokic,26.38,5.0,213.36,250.0
...,...,...,...,...,...,...
356,356,Lance Thomas,6.43,357.0,203.20,240.0
357,357,Alex Abrines,6.28,358.0,198.12,200.0
358,358,Treveon Graham,6.17,359.0,195.58,225.0
359,359,Frank Ntilikina,6.03,360.0,198.12,200.0


In [161]:
import pandas as pd
d = pd.read_csv('full_per_2006.csv')

In [162]:
d['TEAM'] = d['PLAYER'].apply(get_team)
d['PLAYER'] = d['PLAYER'].apply(get_player)

In [163]:
phys = pd.read_csv('nba0506_phys_stats.csv')
data = d.merge(phys, how='left', left_on=['PLAYER'], right_on=['Player'])

In [164]:
data[data.Height.isna()]

,RK,PLAYER,GP_x,MPG,TS%_x,AST_x,TO,USG,ORR,DRR,...,GP_y,PTS,REB,AST_y,NetRtg,OREB%,DREB%,USG%,TS%_y,AST%
151,149.0,Ryan Gomes,61,22.6,0.548,10.9,10.3,14.1,9.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,168.0,Stephen Jackson,81,35.9,0.512,13.0,11.8,21.9,2.0,10.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,NaN,Victor Khryapa,69,21.6,0.516,15.6,14.9,14.0,9.0,15.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,NaN,Ronald Murray,76,27.8,0.487,16.3,11.2,17.5,2.0,6.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,309.0,Darrel Armstrong,62,10.0,0.444,29.6,20.0,15.4,4.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
data.at[151, 'Height'] = '6-7'
data.at[170, 'Height'] = '6-8'
data.at[237, 'Height'] = '6-9'
data.at[240, 'Height'] = '6-3'
data.at[312, 'Height'] = '5-11'

In [166]:
foot = data['Height'].str.extract(r'(\d+)-', expand=False).apply(int)
inches = data['Height'].str.extract(r'-(\d+)', expand=False).apply(int)
data['Height_cm'] = 2.54*(foot * 12 + inches)

In [167]:
data[['PLAYER', 'PER', 'RK', 'Height_cm', 'Weight']].to_csv('short_PER_0506.csv')

In [170]:
data = pd.read_csv('short_PER_0506.csv')
data

,Unnamed: 0,PLAYER,PER,RK,Height_cm,Weight
0,0,Dirk Nowitzki,28.20,1.0,213.36,245.0
1,1,LeBron James,28.17,2.0,203.20,240.0
2,2,Kobe Bryant,28.11,3.0,198.12,220.0
3,3,Dwyane Wade,27.68,4.0,193.04,212.0
4,4,Kevin Garnett,26.88,5.0,210.82,220.0
...,...,...,...,...,...,...
322,322,Mark Madsen,5.26,318.0,205.74,236.0
323,323,Antonio Burks,5.07,319.0,182.88,195.0
324,324,Rafael Araujo,4.51,320.0,210.82,280.0
325,325,Ryan Bowen,3.97,321.0,205.74,220.0
